In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pd-datas/trinary_datasets_only/tqwt_mfccs_concat.csv
/kaggle/input/pd-datas/trinary_datasets_only/mfccs_wavelet_concat.csv
/kaggle/input/pd-datas/trinary_datasets_only/tqwt_mfccs_wavelet.csv
/kaggle/input/pd-datas/trinary_datasets_only/tqwt_wavelet_concat.csv
/kaggle/input/pd-datas/binary_datasets_only/tqwt_concat.csv
/kaggle/input/pd-datas/binary_datasets_only/mfccs_wavelet.csv
/kaggle/input/pd-datas/binary_datasets_only/tqwt_mfccs.csv
/kaggle/input/pd-datas/binary_datasets_only/concat_wavelet.csv
/kaggle/input/pd-datas/binary_datasets_only/mfccs_concat.csv
/kaggle/input/pd-datas/binary_datasets_only/tqwt_wavelet.csv
/kaggle/input/pd-datas/trinary_feature_FH/tqwt_wavelet_concat_FH.csv
/kaggle/input/pd-datas/trinary_feature_FH/tqwt_mfccs_wavelet_FH.csv
/kaggle/input/pd-datas/trinary_feature_FH/tqwt_mfccs_concat_FH.csv
/kaggle/input/pd-datas/trinary_feature_FH/mfccs_wavelet_concat_FH.csv
/kaggle/input/pd-datas/binary_feature_set_FH/mfccs_wavelet_FH.csv
/kaggle/input/pd-dat

In [2]:
## Initial Imports

%matplotlib inline

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import LeaveOneOut, GridSearchCV, KFold, train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, classification_report, matthews_corrcoef, roc_auc_score
from statistics import mean

In [3]:
pdh = pd.read_csv(r'/kaggle/input/pd-datas/trinary_feature_FH/tqwt_wavelet_concat_FH.csv')

print(pdh.head())
#print('The shape of the matrix is :', pdh.shape)
print(list(pdh.columns))

   tqwt_maxValue_dec_14  tqwt_entropy_log_dec_33  tqwt_meanValue_dec_17  \
0              0.105750               -4540.0830           9.600000e-20   
1              0.121580               -4466.8949          -1.180000e-19   
2              0.081818               -4455.2826          -7.420000e-20   
3              0.567740               -2779.8086          -1.070000e-19   
4              0.567490               -2910.9996           1.310000e-19   

   tqwt_meanValue_dec_27  tqwt_entropy_log_dec_35  tqwt_entropy_log_dec_10  \
0           1.840000e-18               -3190.1752             -281439.2690   
1          -4.270000e-18               -3106.4317             -285733.2255   
2          -2.120000e-18               -3082.5691             -305145.0185   
3           5.940000e-19               -1965.3808             -287896.7686   
4           1.280000e-19               -2102.3852             -279129.6866   

   tqwt_meanValue_dec_3  tqwt_entropy_log_dec_28  tqwt_meanValue_dec_6  \
0     

In [4]:
df=pdh
train_df=df.values
x_train=train_df[:,:-1]
y_train=train_df[:,-1]

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train= sc.fit_transform(x_train)
#newly added
kfold=KFold(252, shuffle=False)#random_state=10

In [6]:
def svm_model(X,y, kfold):
    params={'C':[0.1, 1, 10, 100,], 'gamma':[ 0.1, 0.01, 0.001, 0.0001], 'kernel':['rbf']}

    grid=GridSearchCV(svm.SVC(probability=True, random_state=10),params,cv=kfold, refit=True, verbose=1)
    grid.fit(X,y)
    
    return grid

In [7]:
grid=svm_model(x_train, y_train,kfold)

Fitting 252 folds for each of 16 candidates, totalling 4032 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 4032 out of 4032 | elapsed: 27.6min finished


In [8]:
from statistics import mean
print(grid.best_params_)
print(grid.best_score_)

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.83994708994709


In [9]:
wrong_pred_indices=[]
d={}
svm_estimator=svm.SVC(C= 100, gamma= 0.0001, kernel= 'rbf',probability=True, random_state=10)
i=0

correct_ones_=[]#collects correct pd postive predictions
correct_zeros_=[]#collects correct pd negative predictions

wrong_ones_=[]#collect wrong pd positive predictions
wrong_zeros_=[]#collect wrong pd negative predictions

tp=[]# true positive
tn=[]# true negative
fp=[]# false positive
fn=[]# false negative

tridx=[]
tstidx=[]
acc=[]
mcc=[]
f1=[]
tsts=[]

for trids, tstids in kfold.split(x_train):
    
    trainx, testx= x_train[trids,:], x_train[tstids,:]
    trainy, testy= y_train[trids], y_train[tstids]
    
    svm_estimator.fit(trainx, trainy)
    
    preds=svm_estimator.predict(x_train[tstids,:])
    
    acc.append(accuracy_score(testy,svm_estimator.predict(testx)))
    #mcc.append(matthews_corrcoef(testy, svm_estimator.predict(testx)))
    f1.append(f1_score(testy, svm_estimator.predict(testx)))
    
    for i in range(len(tstids)):
        tsts.append(tstids[i])
    
    
    if (i==9):#checking if it is last split
        tridx=trids
        tstidx=tstids

        for j in range(len(tstids)):
            if(preds[j]==0 and (y_train[tstids[j]]==0) ):
                correct_zeros_.append(j)
                tn.append(j)
                
            if(preds[j]==0 and (y_train[tstids[j]]==1) ):
                wrong_zeros_.append(j)
                fn.append(j)
                
            if(preds[j]==1 and (y_train[tstids[j]]==1) ):
                correct_ones_.append(j)
                tp.append(j)
                
            if(preds[j]==1 and y_train[tstids[j]]==0):
                wrong_ones_.append(j)  
                fp.append(j)
    i=i+1

print('acc:', mean(acc))
print('f1:', mean(f1))
#print('mcc:', mean(mcc))


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1465: UndefinedMe

acc: 0.8399470899470899
f1: 0.7138888888888889


In [10]:
j=0
tpids=[]
tpc=0
tnids=[]
tnc=0
fpids=[]
fpc=0
fnids=[]
fnc=0

#performing majority voting
# if accurately predicted over 0.66 for 3 that means we predicted correctly for the whole sample

for t in range(0,len(y_train),3):
    print('t:',t," original:", y_train[t],' patient no: ',j, ' prediction',acc[j] )

    
    if y_train[t]==1 and acc[j]>0.5:
        tpc=tpc+1
        tpids.append(t)
    if y_train[t]==1 and acc[j]<0.5:
        fnc=fnc+1
        fnids.append(t)
    if y_train[t]==0 and acc[j]>0.5:
        tnc=tnc+1
        tnids.append(t)
    if y_train[t]==0 and acc[j]<0.5:
        fpc=fpc+1
        fpids.append(t)
    
    j=j+1

print("true positives: ", tpc)
print("true negatives: ", tnc)
print("fales positives: ", fpc)
print("false negatives: ", fnc)

t: 0  original: 1.0  patient no:  0  prediction 1.0
t: 3  original: 1.0  patient no:  1  prediction 1.0
t: 6  original: 1.0  patient no:  2  prediction 0.6666666666666666
t: 9  original: 1.0  patient no:  3  prediction 1.0
t: 12  original: 1.0  patient no:  4  prediction 1.0
t: 15  original: 1.0  patient no:  5  prediction 1.0
t: 18  original: 1.0  patient no:  6  prediction 1.0
t: 21  original: 1.0  patient no:  7  prediction 0.6666666666666666
t: 24  original: 1.0  patient no:  8  prediction 0.6666666666666666
t: 27  original: 1.0  patient no:  9  prediction 1.0
t: 30  original: 1.0  patient no:  10  prediction 1.0
t: 33  original: 0.0  patient no:  11  prediction 0.6666666666666666
t: 36  original: 1.0  patient no:  12  prediction 1.0
t: 39  original: 1.0  patient no:  13  prediction 1.0
t: 42  original: 0.0  patient no:  14  prediction 0.0
t: 45  original: 1.0  patient no:  15  prediction 1.0
t: 48  original: 1.0  patient no:  16  prediction 1.0
t: 51  original: 1.0  patient no:  1

In [11]:
f1=0
f1=(tpc)/((tpc+0.5*(fpc+fnc)))
print('f1 score:', f1)

f1 score: 0.9137055837563451


In [12]:
import math
mcc=0

mcc=(tpc*tnc-fpc*fnc)/math.sqrt((tpc+fpc)*(tpc+fnc)*(tnc+fpc)*(tnc+fnc))
print('mcc: ',mcc)
precision=tpc/(tpc+fpc)
print('precision: ', precision)
recall=tpc/(tpc+fnc)
print('recall: ',recall)
acc=(tpc+tnc)/(tpc+tnc+fpc+fnc)
print('accuracy: ', acc)

mcc:  0.6211022176401811
precision:  0.8737864077669902
recall:  0.9574468085106383
accuracy:  0.8650793650793651


In [13]:
import lime
import shap

In [14]:
#would have to do lime and shape later